## *Rodar notebook toda segunda-feira pra atualizar informações de transferências, treinadores e principalmente do clube*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db
from datetime import datetime

In [2]:
# Mantendo o padrão de como o sokker mostra as datas nos dados que disponibiliza
agora= str(datetime.now()).split('.')[0]
print("Data:", agora)

Data: 2023-02-24 09:48:45


In [3]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


### Treinadores

In [4]:
url= 'https://sokker.org/xml/trainers.xml'
df_trainers= pega_dados(url, sessao, search= 'trainer')
df_trainers

ID     name   surname countryID age teamID   wage signedContract job  \
0  10207283   Silnei      Bugs        21  70  79085  31800              1   2   
1  10231864  William  Salsicha        21  49  79085  37200              1   1   

  skillStamina skillPace skillTechnique skillPassing skillKeeper  \
0           12        16              7            1           3   
1            9        13             16            8           5   

  skillDefending skillPlaymaking skillScoring skillCoach  
0              7               1            9          7  
1              3              10            5          8

In [5]:
# Inserindo a data de hoje que será importante para acompanharmos suas evoluções
df_trainers= df_trainers.assign(date_extracted= agora)

# Soma de valores nulos e duplicados
nulos= df_trainers.isnull().sum().sum()
duplicados=  df_trainers.duplicated().sum()


if (nulos > 0) or (duplicados > 0):
    print('Há valores nulos ou duplicados. Confira os dados e tente novamente.')
    
else:
    print('Não há valores nulos ou duplicados nos dados. Inserindo dados na base de dados "dados.sokker.db"...')
    
    db= conecta_ao_db('dados_sokker.db')
    curs= db.cursor()
    
    df_trainers.to_sql('trainers', con=db, if_exists='append', index= False)

    print('Dados extraídos com sucesso!')

Não há valores nulos ou duplicados nos dados. Inserindo dados na base de dados "dados.sokker.db"...
Dados extraídos com sucesso!


In [6]:
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()
df = pd.read_sql_query("SELECT * FROM trainers", db)
df

ID       date_extracted     name   surname  countryID  age  teamID  \
0    10206425  2021-07-27 14:46:51  Irapuan     Bilha         21   42   79085   
1    10206425  2021-07-31 09:50:48  Irapuan     Bilha         21   42   79085   
2    10207283  2021-07-31 09:50:48   Silnei      Bugs         21   64   79085   
3    10206425  2021-08-02 09:55:05  Irapuan     Bilha         21   42   79085   
4    10207283  2021-08-02 09:55:05   Silnei      Bugs         21   64   79085   
..        ...                  ...      ...       ...        ...  ...     ...   
122  10231864  2023-02-09 07:20:19  William  Salsicha         21   49   79085   
123  10207283  2023-02-16 10:20:36   Silnei      Bugs         21   70   79085   
124  10231864  2023-02-16 10:20:36  William  Salsicha         21   49   79085   
125  10207283  2023-02-24 09:48:45   Silnei      Bugs         21   70   79085   
126  10231864  2023-02-24 09:48:45  William  Salsicha         21   49   79085   

      wage  signedContract  job  skillStamina  skillPace  skillTechnique  \
0    55500               1    1            13          5              12   
1    55500               1    1            13          5              12   
2    31800               1    2            12         16               7   
3    55500               1    1            13          5              12   
4    31800               1    2            12         16               7   
..     ...             ...  ...           ...        ...             ...   
122  37200               1    1             9         13              16   
123  31800               1    2            12         16               7   
124  37200               1    1             9         13              16   
125  31800               1    2            12         16               7   
126  37200               1    1             9         13              16   

     skillPassing  skillKeeper  skillDefending  skillPlaymaking  skillScoring  \
0               7           11              15               14             1   
1               7           11              15               14             1   
2               1            3               7                1             9   
3               7           11              15               14             1   
4               1            3               7                1             9   
..            ...          ...             ...              ...           ...   
122             8            5               3               10             5   
123             1            3               7                1             9   
124             8            5               3               10             5   
125             1            3               7                1             9   
126             8            5               3               10             5   

     skillCoach  
0            10  
1            10  
2             7  
3            10  
4             7  
..          ...  
122           8  
123           7  
124           8  
125           7  
126           8  

[127 rows x 19 columns]

### Transferências

In [7]:
#Pega informações únicas das transferências já adicionadas para que não sejam inseridos valores duplicados na base
transfers = pd.read_sql_query("SELECT * FROM transfers", db)
dates= transfers['date'].unique()
ids= transfers['ID'].unique()

# Pegando informações das partidas retirando aquelas que já foram pegas
url= 'https://sokker.org/xml/transfers.xml'
df_transfers= pega_dados(url, sessao, search= 'transfer')


#Retirando valores nulos e duplicados 
query= 'date not in @dates and ID not in @ids'
df_transfers.dropna(inplace= True)
df_transfers= df_transfers.query(query)
df_transfers

ID sellerTeamID buyerTeamID sellerTeamName       buyerTeamName  \
0  7160519        49449       79085  De Wittertjes  Organized Chaos FC   

   playerID                 date    price playerValue  
0  38028907  2023-02-20 19:59:25  6854400     1197000

In [8]:
if len(df_transfers) > 0:
    
    # Adiciona dados à tabela
    df_transfers.to_sql('transfers', con=db, if_exists='append', index= False)

In [9]:
transfers = pd.read_sql_query("SELECT * FROM transfers", db)
transfers

ID  sellerTeamID  buyerTeamID      sellerTeamName  \
0   6942634         28049        79085           PlastovFC   
1   6942606         52642        79085         Klucha Team   
2   6942327         95215        79085            ks.śląsk   
3   6942168         43275        79085    Chiles Mexicanos   
4   6942114         50992        79085    Unia Oświęcim 97   
..      ...           ...          ...                 ...   
59  7145038         79085        23311  Organized Chaos FC   
60  7151151         79085        28220  Organized Chaos FC   
61  7150678         79085        63155  Organized Chaos FC   
62  7150675         79085        83155  Organized Chaos FC   
63  7160519         49449        79085       De Wittertjes   

           buyerTeamName  playerID                 date    price  playerValue  
0     Organized Chaos FC  37691056  2021-07-27 17:44:15        4       504000  
1     Organized Chaos FC  37634685  2021-07-27 16:44:07   200000       553000  
2     Organized Chaos FC  37690980  2021-07-26 16:24:46    16000       471000  
3     Organized Chaos FC  37937282  2021-07-25 22:15:10     6000       273000  
4     Organized Chaos FC  37432229  2021-07-25 21:20:21        1       745000  
..                   ...       ...                  ...      ...          ...  
59  Óbudai Goldberger SE  38631417  2023-01-10 17:27:45    11260       655000  
60                 FK MB  37615890  2023-01-27 23:36:04   597732      2317000  
61           Maybato Sur  37432229  2023-01-26 14:10:57    12000      1262000  
62      Random Team Club  37463857  2023-01-26 13:58:39    37000       801000  
63    Organized Chaos FC  38028907  2023-02-20 19:59:25  6854400      1197000  

[64 rows x 9 columns]

In [10]:
len(transfers)

64

### Clube

In [11]:
url= f'https://sokker.org/xml/team-{club_id}.xml'
df_team= pega_dados(url, sessao, search= 'team')

# Inserindo a data de hoje que será importante para acompanharmos evolução do clube
df_team= df_team.assign(date_extracted= agora)

# Retirando colunas que não serão guardadas na base
df_team.drop(columns= ['colShirtKeep', 'colTrausKeep', 'colShirt', 
                       'colTraus', 'colShirt2', 'colTraus2'], inplace= True)

In [12]:
if len(df_team) > 0:
    
    # Adiciona dados à tabela
    df_team.to_sql('teams', con=db, if_exists='append', index= False)

In [13]:
teams = pd.read_sql_query("SELECT * FROM teams", db)
teams

teamID                name  countryID  regionID       dateCreated  \
0    79085  Organized Chaos FC         21       381  2021-07-16 01:49   
1    79085  Organized Chaos FC         21       381  2021-07-16 01:49   
2    79085  Organized Chaos FC         21       381  2021-07-16 01:49   
3    79085  Organized Chaos FC         21       381  2021-07-16 01:49   
4    79085  Organized Chaos FC         21       381  2021-07-16 01:49   
..     ...                 ...        ...       ...               ...   
75   79085  Organized Chaos FC         21       381  2021-07-16 01:49   
76   79085  Organized Chaos FC         21       381  2021-07-16 01:49   
77   79085  Organized Chaos FC         21       381  2021-07-16 01:49   
78   79085  Organized Chaos FC         21       381  2021-07-16 01:49   
79   79085  Organized Chaos FC         21       381  2021-07-16 01:49   

       rank  national                 arenaName    money  trainingType  \
0   1637.12         0  Organized Chaos FC Arena   625431             6   
1   1612.54         0  Organized Chaos FC Arena    97346             6   
2   1748.92         0  Organized Chaos FC Arena   126021             6   
3   1830.62         0  Organized Chaos FC Arena   594557             6   
4   2083.78         0  Organized Chaos FC Arena    97756             6   
..      ...       ...                       ...      ...           ...   
75  4055.22         0  Organized Chaos FC Arena  4113981             8   
76  4067.33         0  Organized Chaos FC Arena  5352585             8   
77  4074.28         0  Organized Chaos FC Arena  6298215             8   
78  3992.05         0  Organized Chaos FC Arena  6570395             8   
79  4030.14         0  Organized Chaos FC Arena   165325             8   

    trainingFormation  fanclubCount  fanclubMood  juniorsMax  \
0                   2           232            6           0   
1                   1           232            5           0   
2                   1           261            6           0   
3                   1           296            6           0   
4                   1           333            6           0   
..                ...           ...          ...         ...   
75                  2          1644            6           0   
76                  2          1662            6           0   
77                  2          1683            4           0   
78                  2          1685            3           0   
79                  2          1695            6           0   

         date_extracted trainingTypeGk trainingTypeDef trainingTypeMid  \
0   2021-07-27 14:46:51           None            None            None   
1   2021-07-31 09:50:48           None            None            None   
2   2021-08-02 09:55:05           None            None            None   
3   2021-08-09 11:32:44           None            None            None   
4   2021-08-16 10:17:59           None            None            None   
..                  ...            ...             ...             ...   
75  2023-01-26 07:21:55              2               6               5   
76  2023-02-02 11:25:26              2               6               5   
77  2023-02-09 07:20:19              2               6               5   
78  2023-02-16 10:20:36              2               6               5   
79  2023-02-24 09:48:45              2               6               5   

   trainingTypeAtt  
0             None  
1             None  
2             None  
3             None  
4             None  
..             ...  
75               5  
76               5  
77               5  
78               5  
79               5  

[80 rows x 19 columns]

In [14]:
len(teams)

80

In [15]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()